In [3]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from matplotlib import pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from tqdm import tqdm
import warnings

from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn import metrics

nastavení

In [4]:
graf_priprava_dat = False

df_path = fr"C:\Users\halik\OneDrive\Dokumenty\GitHub\predikce-nabidky\production code\production dataset.xlsx"#@ cesta k datasetu

#@ promenne pouzite v modelu ktere potrebuji detrendovat, deseasonalizovat...
parametry = ['aktivni-1', 'nove-1', 'zbytek-1'] 
#@ externi regresory arima modelu
external_regressor_list = ["nove-1", "zbytek-1"]

#@ hyperparametry ARIMY 
p = 2  # AR myslim si ze by to melo byt 2 aby nebyl overfitting protoze pacf hodne spadne po lag=2
d = 1  # I 
q = 2  # MA


import dat

In [5]:

#@ args:
    # remove_first: odstrani prvních remove first řádků z datasetu (protože nejsou kompletni...)
    # remove_last: odstrani posledni remove_last řádků proto z data setu 
        # default = 6 (momentalne je poslednich 1 radku na predikce)
def get_df(remove_first = 0, remove_last = 1):
    df = pd.read_excel(df_path)
    df = df.drop(df.index[:remove_first])
    if remove_last != 0:
        df = df.drop(df.index[-remove_last:])
    df['datetime'] = pd.date_range(start='2016-03-01', periods=len(df.index.values), freq='MS')
    return df

detrendování dat

In [6]:

df = get_df(12)
df_detrend = df
columns = ['aktivni', *parametry]
for column in columns:
    df_detrend[column] = pd.Series((df[column]))

deseasonalizace dat

In [7]:
df_deseasoned = df_detrend.copy()
for parametr in parametry:
    decomp = sm.tsa.seasonal_decompose(df_detrend[parametr], period=12, model='additive')
    df_deseasoned[parametr] = df_detrend[parametr] - decomp.seasonal.values

arima modely

In [8]:
df_train = df_deseasoned[:-1]

model = ARIMA(df_train[['aktivni']], order=(p, d, q), exog=df_train[[*external_regressor_list]])
model = model.fit()

C:\Users\halik\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [9]:
df_new = get_df(remove_last = 0) # 0 protze nechci odebrat radky co predpovidam
df_new = df_new.drop(df_new.index[0:-1])
df_pred = model.forecast(exog=df_new[[*external_regressor_list]])
print(f"predikce na dalsi mesic ({int(df_new['datum'])}) je {int(df_pred)} bytu v nabidce")

predikce na dalsi mesic (202304) je 7955 bytu v nabidce


C:\Users\halik\AppData\Local\Temp\ipykernel_17196\3271070792.py:4: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  print(f"predikce na dalsi mesic ({int(df_new['datum'])}) je {int(df_pred)} bytu v nabidce")


    rolling window forecast

In [10]:
# Lists to store predictions and actual values for the current window size
predictions = []
actual_values = []
#! smazat na debuging
warnings.filterwarnings("ignore")
#@ parametres
rw_size = 56
parametry = ['nove-1', 'aktivni']
df_train = df_deseasoned[parametry]
external_regressor_list = ['nove-1']
# predikce pomoci rw forecastu

# Select training data
df_train = df_train[rw_size:]

# Fit ARIMA model
model = ARIMA(df_train[['aktivni']], order=(p, d, q), exog=df_train[[*external_regressor_list]])
model = model.fit()



In [11]:
# Rolling window sizes
rw_sizes = range(1,len(df_train),5)

# Initialize lists to store metrics for each rolling window size
rw_mae_list = []
rw_mse_list = []
rw_rmse_list = []

# Iterate over rolling window sizes
for rw_size in rw_sizes:
    # Lists to store predictions and actual values for the current window size
    predictions = []
    actual_values = []
    
    # Perform rolling window analysis
    for i in range(len(df_train) - rw_size):
        # Select training data
        df_train = df_train[i:i + rw_size]

        # Fit ARIMA model
        model = ARIMA(df_train[['aktivni']], order=(p, d, q), exog=df_train[[*external_regressor_list]])
        model = model.fit()

        # Predict using the model
        df_new = df_train[i + rw_size:i + rw_size + 1]
        df_pred = model.forecast(exog=df_new[[*external_regressor_list]])

        # Extract actual and predicted values
        df_actual = df_train[i + rw_size:i + rw_size + 1]
        df_actual = df_actual[['aktivni']]
        actual_value = df_actual.values.flatten()[0]
        predicted_value = df_pred.item()

        # Append actual and predicted values to respective lists
        actual_values.append(actual_value)
        predictions.append(predicted_value)

    print(f"Iteration {rw_sizes.index(rw_size) + 1} of {len(rw_sizes)} done", end='\r') 
    # Calculate metrics for the current window size
    mae = mean_absolute_error(actual_values, predictions)
    mse = mean_squared_error(actual_values, predictions)
    rmse = np.sqrt(mse)

    # Append metrics to respective lists
    rw_mae_list.append(mae)
    rw_mse_list.append(mse)
    rw_rmse_list.append(rmse)

# Print the metrics for each rolling window size
for i, rw_size in enumerate(rw_sizes):
    print(f"Rolling Window Size: {rw_size}")
    print(f"MAE: {rw_mae_list[i]}")
    print(f"MSE: {rw_mse_list[i]}")
    print(f"RMSE: {rw_rmse_list[i]}")


ValueError: Provided exogenous values are not of the appropriate shape. Required (1, 1), got (0, 1).

In [ ]:
# Predict using the model
df_new = get_df(remove_last = 0) # 0 protze nechci odebrat radky co predpovidam
df_new = df_new.drop(df_new.index[0:-1])
df_pred = model.forecast(exog=df_new[[*external_regressor_list]])
print(f"predikce na dalsi mesic ({int(df_new['datum'])}) je {int(df_pred)} bytu v nabidce")


predikce na dalsi mesic (202304) je 8026 bytu v nabidce


distribuce

In [ ]:
from email.message import EmailMessage
import ssl
import smtplib
import os
import json
#@params

#* Load credentials from config.json
with open('config.json') as config_file:
    config_data = json.load(config_file)

email_user = config_data['email_user']
email_password = config_data['email_password']
email_recievers = config_data['email_recievers']

subject = "predikce nabidky"
email_body = f"""
predikce na dalsi mesic ({str(int(df_new['datum']))[4:]},{str(int(df_new['datum']))[:4]}) je {str(int(df_pred))} bytu v nabidce
"""

def send_mail(email_body):
    odeslat = False
    if odeslat:
        try:
            server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
            server.ehlo()
            server.login(email_user, email_password)
            server.sendmail(email_user, email_recievers, email_text)
            server.close()
            print('Email odeslán! SIIIUUUU')
        except Exception as exception:
            print("Error: %s!\n\n" % exception)
    else:
        print(email_body)
send_mail(email_body)


predikce na dalsi mesic (04,2023) je 8026 bytu v nabidce

